In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import TensorBoard
import tensorflow as tf

os.chdir('../Utils/')
import featureGenerator
from featureGenerator import *
os.chdir('../src/')
import orderbook_lstm
from orderbook_lstm import OrderBookLSTM

Using TensorFlow backend.


# Generate Features and Response Vars

In [2]:
data_dir = '../../ProjectData/'
in_path = data_dir+'msft-orderbook.csv'
out_path = data_dir+'msft-orderbook-all.csv'
out_path2 = data_dir+'msft-orderbook-final.csv'

mergeOrderBookDays(data_dir, out_path, ['msft'])

['msft-20170417-orderbook.csv', 'msft-20170418-orderbook.csv', 'msft-20170419-orderbook.csv', 'msft-20170420-orderbook.csv', 'msft-20170421-orderbook.csv', 'msft-20170424-orderbook.csv', 'msft-20170425-orderbook.csv', 'msft-20170426-orderbook.csv', 'msft-20170427-orderbook.csv', 'msft-20170428-orderbook.csv']
../../ProjectData/msft-20170417-orderbook.csv
../../ProjectData/msft-20170418-orderbook.csv
../../ProjectData/msft-20170419-orderbook.csv
../../ProjectData/msft-20170420-orderbook.csv
../../ProjectData/msft-20170421-orderbook.csv
../../ProjectData/msft-20170424-orderbook.csv
../../ProjectData/msft-20170425-orderbook.csv
../../ProjectData/msft-20170426-orderbook.csv
../../ProjectData/msft-20170427-orderbook.csv
../../ProjectData/msft-20170428-orderbook.csv


In [3]:
createFeatures(out_path, out_path2, response_type = 'Classification')
data = pd.read_csv(out_path2)
data.head()

['direct.bid1', 'direct.bid2', 'direct.bid3', 'direct.bid4', 'direct.bid5', 'direct.bid6', 'direct.bid7', 'direct.bid8', 'direct.bid9', 'direct.bid10']


/Users/adam/Desktop/Stanford/MSE 448/448 Project/Utils/featureGenerator.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['Response'] = response_col


,direct.bid1,direct.bid2,direct.bid3,direct.bid4,direct.bid5,direct.bid6,direct.bid7,direct.bid8,direct.bid9,direct.bid10,...,midPrice_2,midPrice_3,midPrice_4,midPrice_5,midPrice_6,midPrice_7,midPrice_8,midPrice_9,midPrice_10,Response
0,65.03,65.00,64.96,64.95,64.92,64.90,64.89,64.88,64.86,64.85,...,65.095,65.080,65.080,65.095,65.105,65.120,65.120,65.130,65.135,2
1,65.02,65.01,65.00,64.99,64.98,64.97,64.96,64.95,64.92,64.90,...,65.075,65.075,65.080,65.090,65.090,65.095,65.105,65.095,65.125,1
2,65.10,65.09,65.08,65.06,65.05,65.04,65.02,65.01,65.00,64.99,...,65.120,65.120,65.115,65.115,65.115,65.110,65.110,65.110,65.110,0
3,65.10,65.09,65.08,65.06,65.05,65.04,65.03,65.02,65.01,65.00,...,65.120,65.120,65.115,65.115,65.115,65.115,65.115,65.115,65.115,1
4,65.12,65.11,65.10,65.09,65.08,65.06,65.05,65.04,65.03,65.02,...,65.125,65.125,65.125,65.125,65.120,65.120,65.120,65.120,65.120,1


In [ ]:
data = pd.read_csv(out_path2)
data = data.drop(['datetime', 'direct.last_SRO'], axis = 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
cols_to_normalize = [col for col in data.columns if col != 'Response']
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])

dataset = data.values
dataset = dataset.astype('float32')


# Train/Test Split

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=5):
    dataY = get_one_hot(dataset[look_back+1:,dataset.shape[1]-1].astype(int).reshape(-1),3)
    dataX = []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]    
        dataX.append(a)
    return np.array(dataX), np.array(dataY)

In [ ]:
# Convert response variable to one-hot vectors
def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

In [ ]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
print(trainX.shape)
print(trainY.shape)

# Train Model

In [ ]:
tf.reset_default_graph()

timesteps = 10
n_features = 69
n_neurons = 100
n_classes = 3
n_hidden = 1
dropout = None

#lstm = OrderBookLSTM(timesteps, n_neurons, (timesteps,n_features), n_classes, n_hidden, dropout)


lstm = OrderBookLSTM(10, 100, (10,69), 3, 2)

In [ ]:
mod = lstm.get_model()

In [ ]:
import scipy
import scipy.stats as stats
stats.itemfreq(trainX[:,:,68])[0]

In [ ]:
1.4e6

In [ ]:
data['Response'].value_counts()

In [ ]:
# Class weights to change
class_weight = {0 : 1.,
    1: 10.,
    2: 10.} 

mod.fit(trainX, trainY, 
          epochs=10,  
          batch_size=128, 
          verbose=1, 
          class_weight = class_weight)#,
          #callbacks=[TensorBoard(log_dir='Logs/', write_graph=True)])
    

# Make Predictions and get Metrics

In [ ]:
# Training Error
preds_training = mod.predict(trainX).argmax(axis=-1)
pd.Series(preds_training).value_counts()
precision_recall_fscore_support(np.argmax(trainY, axis=1), preds_training)

In [ ]:
# Validation Error
preds = mod.predict(testX).argmax(axis=-1)
pd.Series(preds).value_counts()
precision_recall_fscore_support(np.argmax(testY, axis=1), preds)

# Ignore

In [ ]:
# Class weights to change
class_weight = {0 : 1.,
    1: 15.,
    2: 18.} 

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(10, input_shape=(5,69), return_sequences=False))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(trainX, trainY, 
          epochs=2,  
          batch_size=10, 
          #verbose=2, 
          class_weight = class_weight,
          callbacks=[TensorBoard(log_dir='Logs/testlog', write_graph=True)])

In [ ]:
preds = model.predict(testX).argmax(axis=-1)
pd.Series(preds).value_counts()

In [ ]:
precision_recall_fscore_support(np.argmax(testY, axis=1), preds)